In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2

In [3]:
import sys, os
sys.path.append(os.path.dirname(os.path.join((os.path.pardir), "Modules")))

origin_dir = os.path.join(os.path.pardir, 'Data')


new_dir_path = os.path.join(os.path.pardir, 'Datasets')

true_parasitized_path = os.path.relpath("corrected_images/True_parasitized.csv")
false_parasitized_path = os.path.relpath("corrected_images/False_parasitized.csv")
true_uninfected_path = os.path.relpath("corrected_images/True_uninfected.csv")
false_uninfected_path = os.path.relpath("corrected_images/False_uninfected.csv")

In [4]:
# # RUN THIS CELL IF COLAB

# !pip install faiss-gpu
# !pip install dask[bag]
# !pip3 install tsnecuda==3.0.1+cu113 -f https://tsnecuda.isx.ai/tsnecuda_stable.html

# from tsnecuda import TSNE
# import dask


# # RUN THIS CELL IF COLAB

# !pip install faiss-gpu
# !pip3 install tsnecuda==3.0.1+cu113 -f https://tsnecuda.isx.ai/tsnecuda_stable.html

# from tsnecuda import TSNE

# 

# from google.colab import drive

# drive.mount('/content/gdrive')
# !git clone "https://{pat}@github.com/madhava20217/Malaria-Detection-from-Cells.git"

# !pip install -q -r "/content/Malaria-Detection-from-Cells/requirements_versionless.txt" 

# sys.path.append(os.path.dirname('/content/Malaria-Detection-from-Cells/Modules'))
# origin_dir = "/content/Data"
# new_dir_path = "/content/Datasets/"

# true_parasitized_path  = '/content/Malaria-Detection-from-Cells/EDA/corrected_images/True_parasitized.csv'
# false_parasitized_path = '/content/Malaria-Detection-from-Cells/EDA/corrected_images/False_parasitized.csv'
# true_uninfected_path   = '/content/Malaria-Detection-from-Cells/EDA/corrected_images/True_uninfected.csv'
# false_uninfected_path  = '/content/Malaria-Detection-from-Cells/EDA/corrected_images/False_uninfected.csv'


### Run this cell if you want to download data and label the datapoints

In [5]:
# from Modules.data_download import Data_Download
# from Modules.labelling import Labelling

# download = Data_Download(origin_dir)

In [6]:
parasitized_dir = os.path.join(origin_dir, 'cell-images-for-detecting-malaria', 'cell_images', "Parasitized")
uninfected_dir  = os.path.join(origin_dir, 'cell-images-for-detecting-malaria', 'cell_images', 'Uninfected')

true_parasitized = pd.read_csv(true_parasitized_path, index_col = 0, names = ['ImagePath'], header = 0)
false_parasitized = pd.read_csv(false_parasitized_path, index_col = 0, names = ['ImagePath'], header = 0)

true_uninfected = pd.read_csv(true_uninfected_path, index_col = 0, names = ['ImagePath'], header = 0)
false_uninfected = pd.read_csv(false_uninfected_path, index_col = 0, names = ['ImagePath'], header = 0)

In [7]:
for dataset in [true_parasitized, false_parasitized]:
    dataset['ImagePath'] = dataset['ImagePath'].apply(lambda row: os.path.join(parasitized_dir, row))

for dataset in [true_uninfected, false_uninfected]:
    dataset['ImagePath'] = dataset['ImagePath'].apply(lambda row: os.path.join(uninfected_dir, row))

In [8]:
true_parasitized['status'] = 'true_parasitized'
false_parasitized['status'] = 'false_parasitized'

true_uninfected['status'] = 'true_uninfected'
false_uninfected['status'] = 'false_uninfected'

#joining the dataframes
images = pd.concat([true_parasitized, false_parasitized, true_uninfected, false_uninfected])
images.reset_index(drop = True, inplace = True)

In [9]:
dim1 = []
dim2 = []
for img in true_parasitized['ImagePath']:
    img = cv2.imread(img, 1)[..., ::-1]
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

In [10]:
np.mean(dim1)

134.6656259518733

In [11]:
np.mean(dim2)

133.9325312214438

In [12]:
image_shape = (130,130,3)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
# help(ImageDataGenerator)

In [15]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [17]:
#https://stats.stackexchange.com/questions/148139/rules-for-selecting-convolutional-neural-network-hyperparameters
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its binary so we use sigmoid
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 29, 29, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 64)       0

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

In [20]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [21]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
 
class Root(Tk):
    def __init__(self,model):
        super(Root, self).__init__()
        self.title("Python Tkinter Dialog Widget")
        self.minsize(640, 400)
        self.labelFrame = ttk.LabelFrame(self, text = "Open File")
        self.labelFrame.grid(column = 0, row = 1, padx = 20, pady = 20)
        self.button()
        self.button1() 
        self.model = model
 
    def button(self):
        self.button = ttk.Button(self.labelFrame, text = "Browse A File",command = self.fileDialog)
        self.button.grid(column = 1, row = 1)
  
    def fileDialog(self):
        self.filename = filedialog.askopenfilename(initialdir =  "/", title = "Select A File", filetype =
        (("png files","*.png"),("all files","*.*")) )
        self.label = ttk.Label(self.labelFrame, text = "")
        self.label.grid(column = 1, row = 2)
        self.label.configure(text = self.filename)
 
    def button1(self):
        self.button = ttk.Button(self.labelFrame, text = "submit", command = self.get_prediction)
        self.button.grid(column = 1, row = 20)
        
    def get_prediction(self):
        s=self.model.predict(my_image)
        if(s==[[1.]]):
            self.label.configure(text="non parasitic")
        else:
            self.label.configure(text="parasitic") 
root = Root(model)
root.mainloop()